In [1]:
import numpy as np
import pandas as pd
import requests
import urllib

In [2]:
sightings_df = pd.read_pickle("sightings_merged.pkl")
sightings_df.columns

Index(['Date_Time', 'Shape', 'Duration', 'Summary', 'Posted', 'Detail_Link',
       'Detail_Summary', 'Detail_Summary_nltk', 'Country', 'City', 'State',
       'Location', 'Lat', 'Lon', 'timezone_str', 'Time_UTC', 'sentiment_Vader',
       'sentiment_nltk', 'sentiment_TextBlob', 'objectivity_TextBlob',
       'rel0_2007', 'rel1_2007', 'rel2_2007', 'rel3_2007', 'rel4_2007',
       'rel5_2007', 'par0_2007', 'par1_2007', 'par2_2007', 'par3_2007',
       'rel0_2014', 'rel1_2014', 'rel2_2014', 'rel3_2014', 'rel4_2014',
       'rel5_2014', 'par0_2014', 'par1_2014', 'par2_2014', 'par3_2014',
       'State_num', 'time_num', 'FIPS', 'County'],
      dtype='object')

In [3]:
start_date = '01/01/2015'
end_date = '12/31/2017'
len(sightings_df[(sightings_df["Time_UTC"]>start_date)&(sightings_df["Time_UTC"]<end_date)])

11193

In [4]:
agg_data = sightings_df[(sightings_df["Time_UTC"]>start_date)&(sightings_df["Time_UTC"]<end_date)].value_counts("FIPS")
sightings = agg_data.values
graph_fips = agg_data.index
data_df = pd.DataFrame({"fips": graph_fips, "sightings": sightings})
data_df["fips"] = data_df.fips.astype(str)

In [5]:
import json

# population by county from:
# https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html
# https://api.census.gov/data/2019/pep/population?get=NAME,POP&for=county:*
f = open("resources/county_population.json",)

data = pd.DataFrame(json.load(f))
data.columns = ["name", "population", "state", "county"]
county_population_data = data.iloc[1: , :].copy()
county_population_data["fips"] = county_population_data.state + county_population_data.county
county_population_data.head(5)

,name,population,state,county,fips
1,"Hidalgo County, New Mexico",4198,35,023,35023
2,"Hudson County, New Jersey",672391,34,017,34017
3,"Hunterdon County, New Jersey",124371,34,019,34019
4,"Morris County, New Jersey",491845,34,027,34027
5,"Grant County, New Mexico",26998,35,017,35017


In [6]:
def get_population(fips):
    fips = str(fips).zfill(5)
    p = county_population_data[county_population_data.fips==fips].population
    if len(p) != 1:
        print(f"{fips} not found")
        return None
    
    return p.values[0]

data_df["population"] = data_df["fips"].apply(get_population)
data_df.dropna(axis=0, how="any", inplace=True)
data_df["population"] = data_df["population"].astype(str).astype(int)
data_df["per_cap"] = (data_df["sightings"]/data_df["population"])*1000

In [7]:
no_sightings = county_population_data[county_population_data.fips.isin(data_df.fips)==False].copy()
no_sightings.drop(["name", "state", "county"], axis=1, inplace=True)
no_sightings["per_cap"] = 0
no_sightings["sightings"] = 0

In [8]:
print(f"{len(no_sightings)}, {len(data_df)}")
data_df = data_df.append(no_sightings, ignore_index=True)
print(f"{len(data_df)}")

1518, 1702
3220


In [9]:
import math

def calc_log(c):
    try:
        if c > 0:
            return math.log(c)
    except:
        print(c)
    return None

data_df["per_cap_adj"] = data_df["per_cap"].apply(calc_log)

In [10]:
math.log(data_df[data_df.per_cap!=0].per_cap.max()), math.log(data_df[data_df.per_cap!=0].per_cap.min())

(1.5933517746824761, -7.84772464613074)

In [11]:
min_val = data_df[data_df.per_cap>0].per_cap.min()
max_val = data_df[data_df.fips!='48283'].per_cap.max()
min_val, max_val

(0.00039063980158623193, 2.4752475247524752)

In [12]:
offset = .000001
"""
Counties with no sightings do not show up in this aggregated data. They
were added to the dataset but with a per capita value of 0. However, to
smooth the per capita per 1000 people values for the map, the log of the
value was used. The log of 0 is problematic, so a value just below the
lowest adjusted value in the dataset is used.
"""
data_df.loc[data_df.per_cap==0, ['per_cap_adj']] = math.log(min_val-offset)

"""
Adjusting La Salle County in Texas. Its original per capita per 1000 people
value is just over 21, which is nearly four times the next closest value
of 5.8. The purpose is not to diminish this value as "false" or any such
implication. The purpose of the adjustment is so it does not wash out the
other "high" sighting counties.

https://www.bestplaces.net/people/county/texas/la_salle
"""
adjust_la_salle = True
if adjust_la_salle:
    data_df.loc[data_df.fips=='48283', ['per_cap']] = max_val+offset
    data_df.loc[data_df.fips=='48283', ['per_cap_adj']] = math.log(max_val+offset)

In [13]:
math.log(min_val-offset)

-7.850287831284953

In [14]:
min_adj_val = math.log(min_val-offset)
data_df['per_cap_adj'] = data_df['per_cap_adj'] + abs(min_adj_val)

In [15]:
data_df.sort_values("per_cap", ascending=False).head(5)

,fips,sightings,population,per_cap,per_cap_adj
48,48283,37,7520,2.475249,8.756629
1502,48261,1,404,2.475248,8.756628
1032,32009,2,873,2.290951,8.679255
67,28147,30,14286,2.099958,8.592205
586,49033,4,2483,1.610954,8.327115


In [16]:
data_df.sort_values("per_cap_adj", ascending=False).head(5)

,fips,sightings,population,per_cap,per_cap_adj
48,48283,37,7520,2.475249,8.756629
1502,48261,1,404,2.475248,8.756628
1032,32009,2,873,2.290951,8.679255
67,28147,30,14286,2.099958,8.592205
586,49033,4,2483,1.610954,8.327115


In [17]:
data_df.sort_values("sightings", ascending=False).head(5)

,fips,sightings,population,per_cap,per_cap_adj
0,04013,244,4485414,0.054399,4.938870
1,06037,157,10039107,0.015639,3.692290
2,06073,121,3338330,0.036246,4.532852
3,53033,93,2252782,0.041282,4.662966
4,06059,93,3175692,0.029285,4.319606


In [18]:
data_df.to_pickle("county_sightings.pkl")

In [43]:
agg_data = sightings_df.value_counts("State")
sightings = agg_data.values
graph_fips = agg_data.index
state_sightings = pd.DataFrame({"State": graph_fips, "sightings": sightings})

state_features = ['rel0_2007', 'rel1_2007', 'rel2_2007', 'rel3_2007', 'rel4_2007',
       'rel5_2007', 'par0_2007', 'par1_2007', 'par2_2007', 'par3_2007',
       'rel0_2014', 'rel1_2014', 'rel2_2014', 'rel3_2014', 'rel4_2014',
       'rel5_2014', 'par0_2014', 'par1_2014', 'par2_2014', 'par3_2014']
for state in state_sightings.State.unique():
    state_sightings.loc[state_sightings.State==state, state_features] = sightings_df.loc[sightings_df.State==state, state_features].head(1).values

In [45]:
state_sightings.to_pickle("state_sightings.pkl")